In [2]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [4]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [5]:
# Freezing the base model
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the last classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

In [84]:
df = pd.read_csv('/content/eng.csv')

In [85]:
type(df.iloc[0]['Anger'])

numpy.int64

In [86]:
df

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_b_00001,My Spanish language skills were fairly basic.,0,1,0,0,0
1,eng_train_track_b_00002,Don't mess with my orange juice.,2,0,0,0,0
2,eng_train_track_b_00003,"So, I am from a science background and analyze...",0,0,0,0,0
3,eng_train_track_b_00004,I was writing away.,0,0,1,0,0
4,eng_train_track_b_00005,Apparently it wasn't as life threatening as I ...,0,1,0,1,0
...,...,...,...,...,...,...,...
2763,eng_train_track_b_02764,"""That's were those people went missing"".",0,3,0,1,2
2764,eng_train_track_b_02765,"`` Out of my mouth came, `` I didn't see Creep...",0,2,0,0,1
2765,eng_train_track_b_02766,My muscles all relax With the feeling of an em...,0,0,2,0,0
2766,eng_train_track_b_02767,I was going up to see my Avs play the Red Wings.,0,0,2,0,0


In [87]:
# # List of emotions in the GoEmotions dataset
# emotion_labels = [
#     "admiration", "amusement", "anger", "annoyance", "approval",
#     "caring", "confusion", "curiosity", "desire", "disappointment",
#     "disapproval", "disgust", "embarrassment", "excitement", "fear",
#     "gratitude", "grief", "joy", "love", "nervousness",
#     "optimism", "pride", "realization", "relief", "remorse",
#     "sadness", "surprise", "neutral"
# ]

# # Example: Finding the emotion name for label number 3
# label_number = 3  # Using 0-based index for label number
# emotion_name = emotion_labels[label_number]
# print("The emotion is:", emotion_name)

# Setting the labels of our dataset in the same sequence as above
df['labels'] = df.apply(lambda row:[float(row['Anger']), float(row['Fear']), float(row['Joy']), float(row['Sadness']), float(row['Surprise'])], axis=1)
df = df.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df

,text,labels
0,My Spanish language skills were fairly basic.,"[0.0, 1.0, 0.0, 0.0, 0.0]"
1,Don't mess with my orange juice.,"[2.0, 0.0, 0.0, 0.0, 0.0]"
2,"So, I am from a science background and analyze...","[0.0, 0.0, 0.0, 0.0, 0.0]"
3,I was writing away.,"[0.0, 0.0, 1.0, 0.0, 0.0]"
4,Apparently it wasn't as life threatening as I ...,"[0.0, 1.0, 0.0, 1.0, 0.0]"
...,...,...
2763,"""That's were those people went missing"".","[0.0, 3.0, 0.0, 1.0, 2.0]"
2764,"`` Out of my mouth came, `` I didn't see Creep...","[0.0, 2.0, 0.0, 0.0, 1.0]"
2765,My muscles all relax With the feeling of an em...,"[0.0, 0.0, 2.0, 0.0, 0.0]"
2766,I was going up to see my Avs play the Red Wings.,"[0.0, 0.0, 2.0, 0.0, 0.0]"


In [88]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)  # Use only the desired columns

# Optionally, print the dataset to check
print(dataset)

Dataset({
    features: ['text', 'labels'],
    num_rows: 2768
})


In [90]:
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    # inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]
    label_mapping = {
                          0: 0.0,
                          1: 0.6,
                          2: 0.75,
                          3: 1.0
                      }
    # Apply the mapping to each element in the batch using list comprehension
    inputs["labels"] = torch.tensor([[label_mapping[int(label)] for label in sample] for sample in examples["labels"]])
    # mapped_labels = mapped_labels.to('cuda:0')
    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

Map:   0%|          | 0/2768 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2768
})

In [91]:
print(len(encoded_dataset[0]["labels"]))

5


In [92]:
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [93]:
print(encoded_dataset[45]["labels"])  # Should output a tensor of floats: e.g., tensor([1., 0., 0., ..., 0., 1.])

tensor([0.0000, 0.7500, 0.0000, 0.6000, 0.0000])


In [94]:
dataset = encoded_dataset.train_test_split(test_size=0.2)

In [95]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2214
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 554
    })
})

In [111]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    def assign_score(logits):
      """
      Assigns a label based on the value of the logits.
      """
      if logits < 0.5:
          return 0
      elif 0.5 <= logits < 0.7:
          return 1
      elif 0.7 <= logits < 0.8:
          return 2
      elif 0.8 <= logits <= 1.0:
          return 3
    probs = torch.sigmoid(torch.tensor(logits))
    # Apply assign_score function element-wise using a vectorized approach
    predictions = torch.tensor([[assign_score(logit.item()) for logit in sample] for sample in probs])
    true_labels = torch.tensor([[assign_score(logit.item()) for logit in sample] for sample in labels])


    # Sigmoid is typically used for multi-label classification to get probabilities
    # probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # predictions = torch.round(torch.tensor(logits))
    # Convert probabilities to binary predictions (0 or 1)
    # predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    # f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    # precision = precision_score(labels, predictions, average='macro', zero_division=1)
    # recall = recall_score(labels, predictions, average='macro', zero_division=1)
    # accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy


    # Compute accuracy for each output (class)
    accuracies = []
    for i in range(predictions.shape[1]):  # Iterate over each output
        accuracies.append(accuracy_score(true_labels[:, i], predictions[:, i]))

    # Average accuracy over all outputs
    average_accuracy = np.mean(accuracies)
    # print(f"Average Accuracy: {average_accuracy}")

    # For F1 Score (macro-average), you can calculate the score for each output
    f1_scores = []
    for i in range(predictions.shape[1]):  # Iterate over each output
        f1_scores.append(f1_score(true_labels[:, i], predictions[:, i], average='macro', zero_division=1))

    # Average F1 score over all outputs
    average_f1 = np.mean(f1_scores)


    return {
        "accuracy": average_accuracy,
        "f1": average_f1,
        # "precision": precision,
        # "recall": recall
    }

In [112]:
import torch.nn.functional as F
class ModifiedModel(torch.nn.Module):
    def __init__(self, original_model):
        super(ModifiedModel, self).__init__()
        self.original_model = original_model
        self.fc = torch.nn.Linear(11, 5)  # New FC layer to map 11 -> 5 labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.original_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: [batch_size, 11]
        logits = self.fc(logits)  # Shape: [batch_size, 5]
        predictions = torch.sigmoid(logits)
        # print(predictions)
        # label_mapping = {
        #                   0: 0.0,
        #                   1: 0.6,
        #                   2: 0.75,
        #                   3: 1.0
        #               }
        # # Apply the mapping to each element in the batch using list comprehension
        # mapped_labels = torch.tensor([[label_mapping[int(label)] for label in sample] for sample in labels])
        # mapped_labels = mapped_labels.to('cuda:0')
        # predictions = torch.round(logits)
        if labels is not None:
            # loss = F.cross_entropy(logits, labels.float())
            # loss = F.mse_loss(predictions.float(), labels.float())  # Squared loss
            loss = F.cross_entropy(predictions.float(), labels.float())
            return {"loss": loss, "logits": logits}

        return {"logits": logits}

modified_model = ModifiedModel(model)

In [113]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [114]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.367100,1.379196,0.606137,0.282113
2,1.370100,1.371760,0.602888,0.289810
3,1.364200,1.368712,0.602888,0.284185
4,1.362400,1.365742,0.607581,0.286058
5,1.357200,1.365289,0.604693,0.280684


TrainOutput(global_step=1385, training_loss=1.3641938688092283, metrics={'train_runtime': 748.2172, 'train_samples_per_second': 14.795, 'train_steps_per_second': 1.851, 'total_flos': 0.0, 'train_loss': 1.3641938688092283, 'epoch': 5.0})

In [116]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.344200,1.375350,0.601083,0.278592
2,1.357400,1.370798,0.602888,0.280073
3,1.352700,1.369337,0.606137,0.276368


TrainOutput(global_step=831, training_loss=1.3514186524025082, metrics={'train_runtime': 448.6748, 'train_samples_per_second': 14.804, 'train_steps_per_second': 1.852, 'total_flos': 0.0, 'train_loss': 1.3514186524025082, 'epoch': 3.0})

##Using the trained model to get predictions on the dev set

In [117]:
df_val = pd.read_csv('/content/eng_b.csv')
df_val

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_b_00001,I noticed this months after moving in and doin...,NaN,NaN,NaN,NaN,NaN
1,eng_dev_track_b_00002,can't wait to be in another wedding!,NaN,NaN,NaN,NaN,NaN
2,eng_dev_track_b_00003,Just getting out of the house put a smile on m...,NaN,NaN,NaN,NaN,NaN
3,eng_dev_track_b_00004,Your sister is a heavy sleeper.,NaN,NaN,NaN,NaN,NaN
4,eng_dev_track_b_00005,The police were called.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,eng_dev_track_b_00112,My stomach was hurting.,NaN,NaN,NaN,NaN,NaN
112,eng_dev_track_b_00113,It overflowed and brown shitty diarrhea water ...,NaN,NaN,NaN,NaN,NaN
113,eng_dev_track_b_00114,I got close enough to feel some of the spray f...,NaN,NaN,NaN,NaN,NaN
114,eng_dev_track_b_00115,"I can't breathe right, my head has been stuffe...",NaN,NaN,NaN,NaN,NaN


In [118]:
df_val = df_val.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df_val

,text
0,I noticed this months after moving in and doin...
1,can't wait to be in another wedding!
2,Just getting out of the house put a smile on m...
3,Your sister is a heavy sleeper.
4,The police were called.
...,...
111,My stomach was hurting.
112,It overflowed and brown shitty diarrhea water ...
113,I got close enough to feel some of the spray f...
114,"I can't breathe right, my head has been stuffe..."


In [119]:
val_dataset = Dataset.from_pandas(df_val)  # Use only the desired columns

# Optionally, print the dataset to check
print(val_dataset)

Dataset({
    features: ['text'],
    num_rows: 116
})


In [120]:
def preprocess_function_val(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    # inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]

    return inputs

encoded_dataset_val = val_dataset.map(preprocess_function_val, batched=True)
encoded_dataset_val

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 116
})

In [121]:
encoded_dataset_val.set_format("torch", columns=["input_ids", "attention_mask"])
encoded_dataset_val

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 116
})

In [ ]:
val_training_args = TrainingArguments(
    output_dir="./results",
    # per_device_eval_batch_size=8,
    # num_train_epochs=80,
    # weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)
val_trainer = Trainer(
    model=modified_model,  # model is your trained model
    args=val_training_args,
    tokenizer=tokenizer
)

# Run predictions
predictions = val_trainer.predict(encoded_dataset_val)
pred_logits = predictions.predictions
pred_logits

In [123]:
def assign_score(logits):
      """
      Assigns a label based on the value of the logits.
      """
      if logits < 0.5:
          return 0
      elif 0.5 <= logits < 0.7:
          return 1
      elif 0.7 <= logits < 0.8:
          return 2
      elif 0.8 <= logits <= 1.0:
          return 3
probs = torch.sigmoid(torch.tensor(pred_logits))
# Apply assign_score function element-wise using a vectorized approach
pred_labels = torch.tensor([[assign_score(logit.item()) for logit in sample] for sample in probs])

In [124]:
pred_labels

tensor([[0, 3, 0, 0, 0],
        [0, 0, 3, 0, 0],
        [0, 0, 3, 0, 0],
        [0, 3, 0, 0, 0],
        [0, 3, 0, 0, 3],
        [0, 3, 0, 3, 0],
        [3, 3, 0, 0, 0],
        [0, 0, 3, 0, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 0, 3],
        [0, 3, 0, 0, 3],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 0, 1],
        [0, 3, 0, 0, 0],
        [1, 3, 0, 3, 3],
        [3, 3, 0, 3, 0],
        [0, 3, 3, 0, 0],
        [0, 0, 3, 0, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 0, 0],
        [0, 3, 0, 3, 3],
        [0, 3, 0, 0, 0],
        [0, 3, 0, 3, 0],
        [0, 0, 3, 0, 0],
        [0, 3, 0, 3, 0],
        [0, 0, 3, 0, 0],
        [0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0],
        [0, 0, 3, 0, 1],
        [0, 3, 3, 0, 0],
        [0, 3, 3, 2, 0],
        [0, 3, 0, 0, 1],
        [0, 3, 0, 0, 3],
        [0, 1, 3, 1, 0],
        [0, 3, 3, 0, 0],
        [0, 3, 0, 0, 3],


In [125]:
df_val = pd.DataFrame(pred_labels, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
df_val

,Anger,Fear,Joy,Sadness,Surprise
0,0,3,0,0,0
1,0,0,3,0,0
2,0,0,3,0,0
3,0,3,0,0,0
4,0,3,0,0,3
...,...,...,...,...,...
111,0,3,0,3,0
112,3,3,0,0,0
113,0,3,3,0,0
114,0,3,0,3,0


In [126]:
df_val.insert(loc=0, column='id', value='text')
df_val['id'] = [f'eng_dev_track_b_{i+1:05}' for i in range(len(df_val))]
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_b_00001,0,3,0,0,0
1,eng_dev_track_b_00002,0,0,3,0,0
2,eng_dev_track_b_00003,0,0,3,0,0
3,eng_dev_track_b_00004,0,3,0,0,0
4,eng_dev_track_b_00005,0,3,0,0,3
...,...,...,...,...,...,...
111,eng_dev_track_b_00112,0,3,0,3,0
112,eng_dev_track_b_00113,3,3,0,0,0
113,eng_dev_track_b_00114,0,3,3,0,0
114,eng_dev_track_b_00115,0,3,0,3,0


In [127]:
df_val.to_csv('my_pred_after_FCLayer_and_lastlayer_twitterRoberta.csv', index=False)